## Setup the Experiment

#### Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "47556e8d-33f2-4482-b0b3-30f17ecb271b"

fablib = fablib_manager(project_id=project_id)

In [ ]:
fablib.show_config();

In [ ]:
fablib.verify_and_configure()

In [ ]:
fablib.save_config()

## List resources 

In [ ]:
output_table = fablib.list_sites()

In [ ]:
output_table

In [ ]:
fields=['name','cores_available','ram_available','disk_available','nic_basic_available']

In [ ]:

# output_table = fablib.list_sites(fields=fields)
output_table = fablib.list_sites(output='pandas',fields=fields)

### Filter List

In [ ]:
fablib.list_sites(filter_function=lambda x: x['nic_connectx_5_available'] > 2);

## Create the Experiment Slices

This notebook, will show how to set the following properties on a node:

* Site: Choose the FABRIC site where your node will be hosted.
* Host: Choose the specific worker node that the VM will run on.
* Cores: Number of cores, amount of RAN
* RAM: Amount of RAM
* Disk: Amount of local disk space
* VM Image: The operating system image used for the VM



In [ ]:
slice_name = 'k8s-cluster-slice'
site = fablib.get_random_site()
print(f"Site: {site}")
node1_name = 'Node1'
node2_name = 'Node2'
image = 'default_ubuntu_20'
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
network_name = 'NET1'

In [ ]:
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

### For a single node cluster

In [ ]:
#Create Slice
slice = fablib.new_slice(slice_name)

# Add node
node = slice.add_node(name='Node1', 
                      site='MAX',
                      cores=16, 
                      ram=32, 
                      disk=100, 
                      image='default_ubuntu_20')

#Submit Slice Request
slice.submit()

### For a multi-node cluster

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    node1 = slice.add_node(name=node1_name, site=site, image=image)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Add node
    node2 = slice.add_node(name=node2_name, site=site, image=image)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l3network(name=network_name, interfaces=[iface1, iface2], type='IPv4')
    
    #Submit Slice Request
    slice_id = slice.submit()
    
except Exception as e:
    print(f"{e}")

### Show the slice's attributes

In [ ]:
slice.show();

In [ ]:
slice.list_nodes();

## Delete the slices

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()